---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [203]:
import pandas as pd
import numpy as np

## Read in Data

In [240]:
%%time
train = pd.read_csv("train.csv", encoding = "ISO-8859-1")
test = pd.read_csv("test.csv", encoding = "ISO-8859-1")
addresses = pd.read_csv("addresses.csv")
latlons = pd.read_csv("latlons.csv")

<string>:2: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 2.95 s, sys: 196 ms, total: 3.15 s
Wall time: 4.57 s


In [241]:
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [242]:
addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [243]:
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


## Preprocessing Steps
1. Get the lat/lon features 
2. drop NA-compliance rows from training data
    * also fillna for lon/lat data
3. Find columns to drop:
    * take difference between columns in train and test sets
    * ticket_id is not a feature
    * drop address columns
4. Alternatively, columns to keep:
    * agency_name
    * inspector_name
    * judgment_amount
    * violation_code
    * disposition
    * lat
    * lon
    * possibly generate new column which is difference between ticket_issued_date and hearing_date
5. convert appropriate columns to categorical

In [244]:
# 1. Get lat/lon features
# Join latlons to addresses
full_address = pd.merge(addresses, latlons, on='address', how='left')
# Join to train dataframe
train_data = pd.merge(train, full_address, on='ticket_id', how='inner')

In [245]:
train_data.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance', 'address', 'lat',
       'lon'],
      dtype='object')

In [246]:
# 2. drop rows where compliance = NaN (also lat and lon)
train_data = train_data.dropna(subset=['compliance'])
train_data['lat'].fillna(method='pad', inplace=True)
train_data['lon'].fillna(method='pad', inplace=True)      

train_data['compliance'] = train_data['compliance'].astype(int) # convert compliance data to int

In [247]:
# # 3. columns to drop
# train_drop_cols = list(train.columns.difference(test.columns))
# additional_drop_cols = ['ticket_id', 'violation_street_number', 'violation_street_name', 
#                         'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name',
#                         'city', 'state', 'zip_code', 'non_us_str_code', 'country', 'address']
# drop_cols = train_drop_cols + additional_drop_cols

In [248]:
# 4. columns to keep
train_keep_cols = ['agency_name', 'inspector_name', 'judgment_amount', 'violation_code', 
                   'disposition', 'lat', 'lon']

In [249]:
# 5. convert columns to categorical
# http://pbpython.com/categorical-encoding.html
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_data['agency_name'] = le.fit_transform(train_data["agency_name"])
train_data['inspector_name'] = le.fit_transform(train_data["inspector_name"])
train_data['violation_code'] = le.fit_transform(train_data["violation_code"])
train_data['disposition'] = le.fit_transform(train_data["disposition"])

In [250]:
X = train_data[train_keep_cols]
y = train_data['compliance']
print(X.shape)
print(y.shape)


(159880, 7)
(159880,)


In [251]:
X.dtypes

agency_name          int64
inspector_name       int64
judgment_amount    float64
violation_code       int64
disposition          int64
lat                float64
lon                float64
dtype: object

In [252]:
# ensure no null values
X.isnull().values.any()

False

## Fit Gradient Boosting Classifiers and Evaluate

In [253]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [256]:
%%time
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier().fit(X_train, y_train)

CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s
Wall time: 11.4 s


In [257]:
pred_probas = clf.predict_proba(X_test)[:, 1]

In [258]:
from sklearn.metrics import auc, roc_curve, accuracy_score
# Calculate Area Under the (Receiver Operating Characteristic) Curve
fpr, tpr, threshold = roc_curve(y_test, pred_probas)
roc_auc = auc(fpr, tpr)

print(roc_auc)

0.804469114006


In [259]:
%%time
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_train, y_train, scoring="roc_auc", cv=5)
print(scores)
print(scores.mean()) # cross-validated score

[ 0.80912058  0.79372945  0.80902666  0.80836182  0.80674437]
0.805396574587
CPU times: user 33.9 s, sys: 8 ms, total: 33.9 s
Wall time: 48.3 s


## Use test data for submission

In [263]:
def blight_model():
    ## Preprocessing
    # 1. Get lat/lon features
    full_address = pd.merge(addresses, latlons, on='address', how='left') # Join latlons to addresses
    test_data = pd.merge(test, full_address, on='ticket_id', how='inner') # Join to test dataframe
    # 2. drop NA rows for lat and lon - s
    test_data['lat'].fillna(method='pad', inplace=True)
    test_data['lon'].fillna(method='pad', inplace=True)  
    # 4. columns to keep
    test_keep_cols = ['agency_name', 'inspector_name', 'judgment_amount', 'violation_code', 
                       'disposition', 'lat', 'lon']
    # 5. convert columns to categorical
    le = LabelEncoder()
    test_data['agency_name'] = le.fit_transform(test_data["agency_name"])
    test_data['inspector_name'] = le.fit_transform(test_data["inspector_name"])
    test_data['violation_code'] = le.fit_transform(test_data["violation_code"])
    test_data['disposition'] = le.fit_transform(test_data["disposition"])
    X = test_data[test_keep_cols]
    # print(X.shape)
    # print(X.isnull().values.any() == False) # ensure no null values
    
    # Make predictions and Reindex
    test_proba = clf.predict_proba(X)[:, 1]
    test_data['compliance'] = test_proba
    test_data.set_index('ticket_id', inplace=True)
    
    return test_data.compliance # Your answer here

In [264]:
# blight_model()

In [265]:
# bm = blight_model()
# res = 'Data type Test: '
# res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
# res += 'Data shape Test: '
# res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
# res += 'Data Values Test: '
# res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
# res += 'Data Values type Test: '
# res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
# res += 'Index type Test: '
# res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
# res += 'Index values type Test: '
# res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(bm.index[0])).count("int64")>0]

# res += 'Output index shape test:'
# res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][bm.index.shape==(61001,)]

# res += 'Output index test: '
# if bm.index.shape==(61001,):
#     res +=['Failed\n','Passed\n'][all(pd.read_csv('test.csv',usecols=[0],index_col=0).sort_index().index.values==bm.sort_index().index.values)]
# else:
#     res+='Failed'
# print(res)